<a href="https://colab.research.google.com/github/LLAES07/SQL_DESAFIO/blob/main/Caso_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<b><center><h1>DESAFIO N°1 - 8 SEMANAS DE SQL</h1></center></b>



# **IMPORTAR LIBRERIAS NECESARIAS**

In [1]:
import sqlite3
import pandas as pd
import numpy as np

# **GENERACIÓN BASE DE DATOS**

In [2]:
# Generando una conexción a una nueva base de datos

connection = sqlite3.connect('test.db')

# Generando el objeto cursos para poder interactuar con la base de datos

cursor = connection.cursor()

# **GENERACIÓN DE LAS TABLAS**

## **TABLA SALES**

In [3]:

# Generando la tabla sales
cursor.execute("""
  CREATE TABLE sales (
  "customer_id" VARCHAR(1),
  "order_date" DATE,
  "product_id" INTEGER
)

""")

# Generando el commit para actualizar
connection.commit()

In [4]:
# Comprobando la existencia de la tabla
pd.read_sql_query("SELECT * FROM sales", connection)

,customer_id,order_date,product_id


In [5]:
# Generando una lista con los valores para poblar la tabla
fill_list_sales = [
  ('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3')
]

# Utilizando executemany para poder ejecutar de manera rápida y eficiente el poblado de la tabla
cursor.executemany(
    """INSERT INTO sales (customer_id, order_date, product_id) VALUES (?, ?, ?)""", fill_list_sales)

# Actualizando los datos
connection.commit()

## **TABLA MENU**

In [6]:
# Generación Tabla Menu
cursor.execute("""
                    CREATE TABLE IF NOT EXISTS menu(
                    product_id INTEGER, product_name VARCHAR(5), price INTEGER)""")

connection.commit()


# Generando una lista con los valores para poblar la tabla
fill_list_menu = [
    ('1', 'sushi', '10'),
    ('2', 'curry', '15'),
    ('3', 'ramen', '12')
]

# Utilizando executemany para poder ejecutar de manera rápida y eficiente el poblado de la tabla
cursor.executemany(
    "INSERT INTO menu (product_id, product_name, price) VALUES (?, ?, ?)", fill_list_menu)

# Actualizando los datos
connection.commit()




## **TABLA MEMBERS**

In [7]:
# Creando tabla members

cursor.execute("""
  CREATE TABLE IF NOT EXISTS members (
  "customer_id" VARCHAR(1),
  "join_date" DATE
)
""")


# Generando la actualización
connection.commit()


# Generando una lista con los valores para poblar la tabla
fill_list_members = [
  ('A', '2021-01-07'),
  ('B', '2021-01-09')
]


# Utilizando executemany para poder ejecutar de manera rápida y eficiente el poblado de la tabla
cursor.executemany(
    """
    INSERT INTO members (customer_id, join_date) VALUES (?, ?)
    """, fill_list_members
)

# Actualizando los datos
connection.commit()

# **Revisando las tablas**

In [8]:
pd.read_sql_query("SELECT * FROM sales", connection)

,customer_id,order_date,product_id
0,A,2021-01-01,1
1,A,2021-01-01,2
2,A,2021-01-07,2
3,A,2021-01-10,3
4,A,2021-01-11,3
5,A,2021-01-11,3
6,B,2021-01-01,2
7,B,2021-01-02,2
8,B,2021-01-04,1
9,B,2021-01-11,1


In [9]:
pd.read_sql_query("SELECT * FROM menu", connection)

,product_id,product_name,price
0,1,sushi,10
1,2,curry,15
2,3,ramen,12


In [10]:
pd.read_sql_query("SELECT * FROM members", connection)

,customer_id,join_date
0,A,2021-01-07
1,B,2021-01-09


## Entendiendo las relaciones de las tablas

**Sales** se relaciona con **menu** con `product_id` que corresponde al id de cada producto. Por lo que su relación es de **n:1**.


Además **Sales** tambien se relaciona con la tabla **members** por la clave foreana `customer id` en donde la relación es **n:1.**



---



/* --------------------
   Case Study Questions
   --------------------*/

**-- 1. Cual es la cantidad total de dinero que cada cliente ha gastado en el restaurant?**


-- Cantidad total de cada usario gastado en el restaurant

-- customer_id, total_spend

In [11]:
query ="""
      SELECT
        s.customer_id,
        SUM(m.price) AS total_gastado
       FROM
            sales s
      LEFT JOIN menu m
        ON s.product_id = m.product_id
      GROUP BY s.customer_id
"""

pd.read_sql_query(query, connection)


,customer_id,total_gastado
0,A,76
1,B,74
2,C,36


**Explicación**: Necesitamos obtener la cantidad gastada por cada usuario. En nuestra tabla sales tenemos todas las compras realizadas y ademas el id de cada producto.

  Tenemos:

    - El registro de cada compra
  
  Nos falta:

    - El precio de cada objeto comprado


Para poder obtener los precios de cada compra necesitamos hacer un join entre **sales** y **menu** que es donde se encuentran los valores de cada producto comprado en la tabla sales

**-- 2. Cuantos días cada cliente ha visitado el restaurant?**


-- Identificar que tabla tiene el dato de visita al restaurant

-- Contar la cantidad de dias segun cada usuario

In [12]:

query = """

  SELECT
    customer_id,
    COUNT(DISTINCT order_date) AS days_visited
  FROM
    Sales
  GROUP BY
    customer_id


"""
pd.read_sql_query(query, connection)

,customer_id,days_visited
0,A,4
1,B,6
2,C,2


**Explicación**: Una vez identificado que la tabla **sales** tiene la información de cada compra del usuario además de la **fecha en la cual realizó la compra** el problema se simplifica.
Agrupar por usuario y **contar cada fecha unica como 1 día** que ha sido visitado

**-- 3. Cúal fue el primer item del menu comprado por cada cliente?**


In [13]:
query ="""
  -- Tabla temporal para rankear las compras de cada usuario

  WITH primera_orden AS (

    SELECT
      s.customer_id,
      m.product_name,
      s.order_date,
      DENSE_RANK() OVER (PARTITION BY s.customer_id ORDER BY s.order_date ASC) AS ranking
    FROM
      sales s
    LEFT JOIN menu m
      ON s.product_id = m.product_id
  )

  -- Tabla final con el primer producto comprado por cada usuario

  SELECT
    customer_id,
    product_name
  FROM
    primera_orden
  WHERE
    ranking = 1


"""

pd.read_sql_query(query, connection)

,customer_id,product_name
0,A,sushi
1,A,curry
2,B,curry
3,C,ramen
4,C,ramen


**-- 4. Cuál es el item más comprado en el menu y cuantas veces fue comprado por todos los usarios?**


In [19]:
query = """

-- Contador de productos

WITH count_products  AS  (
  SELECT
  product_name, COUNT(product_name) AS n_times_sold
  FROM sales s
  LEFT JOIN menu m
  ON s.product_id = m.product_id
  GROUP BY product_name ),


-- Ranking de productos
 ranking_products AS (
 	SELECT
   		*,
   		DENSE_RANK() OVER (ORDER BY n_times_sold DESC) AS ranking
   FROM count_products)

-- Seleccionar el numero del ranking

 SELECT
 	product_name,
  n_times_sold
  FROM
 	 ranking_products
 WHERE ranking = 1;

"""

pd.read_sql_query(query, connection)


,product_name,n_times_sold
0,ramen,8


**Explicación**: El primer paso es saber cuantas veces se han comprado cada producto. El siguiente consiste en generar un ranking que sea facilmente actualizable usando DENSE_RANK(). Con esta segunda tabla temporal podremos filtrar por aquel item que tenga la mayor cantidad de unidades vendidas.
Por ultimo se filtra la tabla del ranking para encontrar cual es el item mas vendido.

**-- 5. Cual item fue el más popular para cada usuario ?**


In [20]:
query = """

-- Tabla temporal con la cantidad de veces que se ha comprado cada producto

WITH mostsolds_by_customer AS (
 SELECT
 	  customer_id,
  	product_name,
    COUNT(product_name) AS n_times_sold
  FROM sales s
  LEFT JOIN menu m
  ON s.product_id = m.product_id
  GROUP BY customer_id ,product_name ),

-- ranking de productos por usuario
ranking_pop AS (

  SELECT
  	*,
  	DENSE_RANK() OVER (PARTITION BY customer_id ORDER BY n_times_sold DESC) AS ranking

  FROM
  	mostsolds_by_customer

)

-- Seleccionar el ranking 1 de cada cliente

SELECT
	customer_id,
   	product_name
FROM ranking_pop
WHERE ranking = 1

"""

pd.read_sql_query(query, connection)

,customer_id,product_name
0,A,ramen
1,B,curry
2,B,ramen
3,B,sushi
4,C,ramen


**Explicación:** Similar al anterior pero ahora incluyendo la partición por cliente generamos un conteo de productos según cada cliente. Una vez generado esto lo dejamos en una tabla temporal. Usando la tabla temporal generamos un raking asegurandonos de que la partición este sobre cada cliente para que asi cada ranking de conteo de productos se asocie con su respectivo cliente. Ya generado el ranking tomamos esta tabla temporal y la filtramos según el numero del ranking que queramos en este caso 1 y tenemos los items que son los mas vendidos por cada usuario. En nuestro caso el cliente B ha comprado la misma cantidad tanto de curry, ramen y sushi.

**-- 6. Que item fue comprado primero por cada cliente después que se convitieron en miembro?**

In [21]:
query = """

  WITH first_buy_after_membership AS (
  SELECT
      s.customer_id,
          s.product_id,
          s.order_date,
          DENSE_RANK() OVER(PARTITION BY s.customer_id ORDER BY s.order_date) AS ranking
  FROM sales s
  LEFT JOIN members m
  ON s.customer_id = m.customer_id AND s.order_date >= m.join_date
  WHERE m.customer_id IS NOT NULL )

  SELECT
    f.customer_id,
      m.product_name
  FROM
    first_buy_after_membership f
  INNER JOIN menu m
  ON f.product_id = m.product_id
  WHERE ranking=1


"""

pd.read_sql_query(query, connection)

,customer_id,product_name
0,B,sushi
1,A,curry


**Explicación**: Según lo que nos pide el ejercicio sabemos que necesitamos la tabla sales y members primeramente, ya que necesitamos encontrar aquellos productos comprados después de ser miembros. En la primera tabla temporal `first_buy_after_membership` la idea es filtrar la tabla **sales** para saber cuales son los productos que fueron comprado primeramente. Con un LEFT JOIN nos aseguramos que se incluyan todas las fechas y sus items comprados respectivamente. Una vez realizado esto necesitamos que para cada custumer se junte latabla según el customer_id peero ademas según el _order_date_ el cual debe ser mayor a *m.join_date*. Con esto nos aseguramos que la compra del producto sea mayor o igual a la fecha de que se hizo miembro. Además con esta tabla podemos generar un ranking en donde el primer registro será el más viejo. Ya con esto se puede filtrar por el ranking = 1.

**-- 7. Cúales items fueron comprados justo antes de que el cliente se convirtiera en miembro?**

In [22]:
query = """


  WITH first_buy_before_membership AS (
  SELECT
    s.customer_id,
        s.product_id,
        s.order_date,
        DENSE_RANK() OVER(PARTITION BY s.customer_id ORDER BY s.order_date DESC) AS ranking
  FROM sales s
  LEFT JOIN members m
  ON s.customer_id = m.customer_id AND s.order_date < m.join_date
  WHERE m.customer_id IS NOT NULL )

  SELECT
  f.customer_id,
    m.product_name
  FROM
  first_buy_before_membership f
  INNER JOIN menu m
  ON f.product_id = m.product_id
  WHERE ranking=1


"""

pd.read_sql_query(query, connection)

,customer_id,product_name
0,A,sushi
1,B,sushi
2,A,curry


**Explicación**: Similar al anterior ahora nos aseguramos de generar una tabla temporal en donde nos aseguremos que la fecha de que se unen como miembro sea mayor a la de la orden, de esta manera tendremos todos los pedidos que fueron comprados antes de convertirse en miembros. Por ultimo realizamos un filtrado de la tabla demporal para saber cual es el producto comprado justo antes de convertirse en miembro de cada usuario. Como C no es miembro no está incluido.

**-- 8. Cúal es el total de items y la cantidad gastada por cada miembro antes de que se convirtieran en miembros?**


In [37]:
query = """

-- Tabla temporal con los productos antes de convertirse en miembros

WITH first_buy_before_membership AS (
SELECT
	  s.customer_id,
        s.product_id,
        s.order_date
FROM sales s
LEFT JOIN members m
ON s.customer_id = m.customer_id AND s.order_date < m.join_date
WHERE m.customer_id IS NOT NULL )


SELECT
	f.customer_id,
    COUNT(m.product_name) AS total_items,
    SUM(m.price) AS total_spend
FROM first_buy_before_membership f
INNER JOIN menu m
ON f.product_id = m.product_id
GROUP BY f.customer_id



"""

pd.read_sql_query(query, connection)

,customer_id,total_items,total_spend
0,A,2,25
1,B,3,40


Explicación: Juntamos la tabla **Sales** con la tabla **miembos** ya que esta tiene la fecha en donde comenzaron a hacer miembros los usuarios. Gracias a esto ponemos la condición de que la fecha sea menor a la fecha de inicio de la membresia. Con esto generamos otro cruce pero ahora con **menu** para saber cuales son los precios de cada producto y asi poder sacar el total gastado.

**-- 9.  Si cada $1 gastado es igual a 10 puntos y el sushi tiene un multiplicador de 2x - Cuantos puntos deberia tener cada cliente ?**

In [33]:
query = """

SELECT
	customer_id,
	SUM(CASE WHEN product_name = 'sushi' THEN price * 20 ELSE price*10 END) AS puntos_clientes
FROM sales s
LEFT JOIN menu m
USING (product_id)
GROUP BY customer_id


"""

pd.read_sql_query(query, connection)

,customer_id,puntos_clientes
0,A,860
1,B,940
2,C,360


**Explicación: **Según lo explicado sabemos que si cada 1$ = 10 pts sabemos que para el **sushi 1\$ = 20 osea 20 x price**. Con esto ya es facil utilizar un CASE WHEN para cuando el product_name sea sushi multiplicar por 20 y en cualquier otro caso multiplicar por 10. Con esto además nos aseguramos juntar la tabla sales con menu ya que en estas se encuentran los precios de cada items necesarios para calcular los puntos. Agrupamos por clientes y tenemos el resultado.

**-- 10. En la primera semana después que un cliente se una al programa (incluyendo la fecha de ingreso) estos ganan 2x puntos en todos los items, no solo en sushi- Cuantos puntos tienen el cliente A y B para el final de enero ?**


In [35]:
query = """

-- Tabla temporal con la fecha desde el inicio hasta 1 semana despues de la fecha de inicio de membresia

WITH time_interval AS (
SELECT
  	customer_id,
	join_date as start_date,
    date(join_date, '+7 days') as end_date
FROM members )


-- Utilizando la fecha de inicio y final de la semana para filtrar las fechas y multiplicar segun si está dentro o fuera de esta fecha

SELECT
	s.customer_id,
	SUM(CASE WHEN t.start_date IS NOT NULL THEN 20*price ELSE 10*price END) AS puntos
FROM sales s
LEFT JOIN time_interval t
ON s.customer_id = t.customer_id AND s.order_date BETWEEN t.start_date AND t.end_date
LEFT JOIN menu m
ON s.product_id = m.product_id
GROUP BY s.customer_id



"""

pd.read_sql_query(query, connection)

,customer_id,puntos
0,A,1270
1,B,960
2,C,360


**Explicación:** El primer paso es la creación de una tabla temporal que tenga dos columnas con el valor de comienzo de la semana de membresia y el final de la semana de membresia. Una vez con esta tabla temporal se utiliza para juntarla con la tabla **sales** y **menu**, asegurandose que la orden de la fecha de cada item este entre el rango de la fecha calculada en la tabla temporal. Realizado este paso tendremos todas las compras y aquellas que no tengan valores nulos en sus joins se multiplican por 20 porque representan a ordenes generadas en al semana de inicio de membresia y por 10 cuando no estan dentro de la semana de membresia.

**PD:** Notar que no se incluyen más filtros como que sea en enero y que sean los custumers A Y B que según los demás ejercicios está claro que solo A Y B son miembros y además los datos son todos de enero.